In [1]:
import pandas as pd

In [2]:
tpm = pd.read_csv('16p12_lcl_gene_tpm.by_subject.50perc.gct', sep='\t')
gtf = pd.read_csv('../eqtl/gencode.v19.genes.v7.patched_contigs.with_chr.gtf', sep='\t', header=None)

In [3]:
mapp = pd.read_csv('../16p12.2_rnaseq_analysis/data/gene_names_mapping_new.tsv', sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
mapp = mapp.set_index('ensembl', drop=False)
tpm = tpm.set_index('ensembl', drop=False)

In [5]:
gtf['ensembl'] = gtf[8].apply(lambda s : s.split('"')[1])
gtf['ensembl'] = gtf['ensembl'].apply(lambda s: s.split('.')[0])
gtf = gtf.set_index('ensembl', drop=False)
gtfs = gtf.drop_duplicates('ensembl')

In [6]:
tpm_new = mapp[['chromosome', 'start', 'end', 'ensembl']].copy()
tpm_new['name2'] = tpm_new['ensembl']
tpm_new['strand'] = tpm_new.ensembl.apply(lambda s: gtfs.at[s, 6])

In [7]:
tpm_new = tpm_new.loc[tpm.index]

In [8]:
for subject in tpm.columns[1:]:
#     print(subject)
    tpm_new[subject] = tpm[subject]

In [9]:
tpm_new.columns = ['#chromosome'] + tpm_new.columns[1:].tolist()
tpm_new = tpm_new.sort_values(['#chromosome','start'])

In [10]:
tpm_new['#chromosome'] = tpm_new['#chromosome'].apply(lambda s: 'chrM' if s == 'MT' else 'chr' + str(s))

In [11]:
tpm_new.to_csv('16p12_lcl_gene_tpm.by_subject.50perc.anno.bed', sep='\t', index=False)